In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import pandas as pd
import glob
from collections import Counter

# 경로 설정
training_image_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_image'
training_label_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_label'
validation_image_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_image'
validation_label_path = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_label'

## 2-1. training, validatiron 통계표 작성

In [ ]:
from concurrent.futures import ProcessPoolExecutor

# 유효한 이미지 파일 목록 생성 함수 (Training과 Validation)
def get_valid_image_ids(image_path):
    valid_ids = {os.path.splitext(f)[0].split('_')[1] for f in os.listdir(image_path) if f.endswith('.jpg')}
    print(f"{image_path}에서 유효한 이미지 ID 수: {len(valid_ids)}")  # 디버그 출력
    return valid_ids

# 병렬 처리 함수 (전역 함수로 정의)
def process_file(file_path, valid_image_ids):
    file_name = os.path.basename(file_path)
    try:
        # 파일명에서 필요한 정보를 추출
        parts = file_name.split('_')
        image_id = parts[1]   # 이미지 ID
        style = parts[3]      # 스타일
        gender = parts[4]     # 성별

        # 유효한 이미지 ID만 처리
        if image_id in valid_image_ids:
            # 성별 & 스타일 조합별 설문ID 카운트 증가
            return (gender, style)
    except IndexError:
        print(f"파일 {file_name} 처리 중 오류 발생")
    return None

# 라벨링 데이터를 읽고 유효한 데이터에 대해 성별, 스타일별 설문ID 수 통계를 작성하는 함수 (병렬 처리)
def collect_stats(label_path, valid_image_ids):
    stats = Counter()

    # glob을 이용해 모든 .json 파일 검색
    json_files = glob.glob(os.path.join(label_path, "*.json"))
    print(f"{label_path}에서 JSON 파일 수: {len(json_files)}")  # 디버그 출력

    # ProcessPoolExecutor를 사용하여 파일 병렬 처리
    with ProcessPoolExecutor() as executor:
        results = executor.map(process_file, json_files, [valid_image_ids] * len(json_files))

    # 결과 통합
    for result in results:
        if result:
            stats[result] += 1

    return stats

# 유효한 이미지 파일 ID 목록 생성 (Training과 Validation)
training_image_ids = get_valid_image_ids(training_image_path)
validation_image_ids = get_valid_image_ids(validation_image_path)

# Training 및 Validation 데이터에 대한 통계 수집
print("Training 데이터 통계 수집 중...")
training_stats = collect_stats(training_label_path, training_image_ids)

print("Validation 데이터 통계 수집 중...")
validation_stats = collect_stats(validation_label_path, validation_image_ids)

# 결과를 데이터프레임으로 변환하여 보기 좋게 출력
def stats_to_dataframe(stats):
    df = pd.DataFrame(
        [(gender, style, count) for (gender, style), count in stats.items()],
        columns=["성별", "스타일", "이미지수"]
    )
    # 성별을 기준으로 M, W 순서로 정렬
    df = df.sort_values(by=["성별", "스타일"], key=lambda x: x.map({'M': 0, 'W': 1})).reset_index(drop=True)
    return df

# 데이터프레임 생성 및 출력
training_df = stats_to_dataframe(training_stats)
validation_df = stats_to_dataframe(validation_stats)

/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_image에서 유효한 이미지 ID 수: 4066
/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_image에서 유효한 이미지 ID 수: 951
Training 데이터 통계 수집 중...
/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_label에서 JSON 파일 수: 211346
Validation 데이터 통계 수집 중...
/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_label에서 JSON 파일 수: 36382


In [ ]:
# training 데이터프레임
training_df

,성별,스타일,이미지수
0,M,sportivecasual,1657
1,M,normcore,1096
2,M,metrosexual,1226
3,M,bold,1144
4,M,hippie,1571
5,M,hiphop,1321
6,M,ivy,1719
7,M,mods,1616
8,W,normcore,646
9,W,genderless,234


In [ ]:
# validation 데이터프레임
validation_df

,성별,스타일,이미지수
0,M,ivy,537
1,M,hiphop,259
2,M,hippie,474
3,M,bold,221
4,M,mods,438
5,M,metrosexual,224
6,M,sportivecasual,180
7,M,normcore,102
8,W,sportivecasual,278
9,W,lounge,20


## 2.2 “스타일 선호 정보표 구하기

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# 유효한 이미지 파일 목록 생성 함수 (Training과 Validation)
def get_valid_image_ids(image_path):
    valid_ids = {os.path.splitext(f)[0].split('_')[1] for f in os.listdir(image_path) if f.endswith('.jpg')}
    print(f"{image_path}에서 유효한 이미지 ID 수: {len(valid_ids)}")  # 디버그 출력
    return valid_ids

# 유효한 라벨링 파일 목록 (이미지 ID 목록에 맞춰 필터링)
def get_valid_labels(label_path, valid_image_ids):
    files = glob.glob(os.path.join(label_path, "*.json"))
    valid_labels = [label for label in files if os.path.basename(label).split('_')[1] in valid_image_ids]
    print(f"유효한 라벨 파일 수 ({label_path}): {len(valid_labels)}")  # 디버그 출력
    return valid_labels

# 유효한 이미지 ID 가져오기
training_image_ids = get_valid_image_ids(training_image_path)
validation_image_ids = get_valid_image_ids(validation_image_path)

# 유효한 라벨 파일 목록 생성
training_labels = get_valid_labels(training_label_path, training_image_ids)
validation_labels = get_valid_labels(validation_label_path, validation_image_ids)

# 각 파일을 처리하여 응답 데이터를 수집하는 함수
def process_label_file(json_file, label_type):
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
            user_id = data['user']['R_id']
            style = data["item"]["imgName"]
            prefer = data["item"]["survey"]["Q5"]

            # 각 사용자 응답 정보 딕셔너리 생성
            user_data = {
                "user_id": user_id,
                "label_type": label_type,
                "like": [style] if prefer == 2 else [],
                "hate": [style] if prefer == 1 else []
            }
            return user_data
    except Exception as e:
        print(f"Error processing {json_file}: {e}")
        return None

# 응답자 선호 정보 수집 함수 (병렬 처리)
def collect_user_preferences_parallel(training_labels, validation_labels, max_users=100):
    user_data = {}

    # ThreadPoolExecutor를 사용하여 병렬로 파일 처리
    with ThreadPoolExecutor() as executor:
        # Training과 Validation을 병렬로 처리
        futures = [
            executor.submit(process_label_file, json_file, "training") for json_file in training_labels
        ] + [
            executor.submit(process_label_file, json_file, "validation") for json_file in validation_labels
        ]

        for future in as_completed(futures):
            result = future.result()
            if result:
                user_id = result["user_id"]
                label_type = result["label_type"]

                if user_id not in user_data:
                    user_data[user_id] = {
                        "training_like": [],
                        "training_hate": [],
                        "validation_like": [],
                        "validation_hate": [],
                        "response_count": 0
                    }

                # Training과 Validation 선호 정보를 구분하여 저장
                if label_type == "training":
                    user_data[user_id]["training_like"].extend(result["like"])
                    user_data[user_id]["training_hate"].extend(result["hate"])
                elif label_type == "validation":
                    user_data[user_id]["validation_like"].extend(result["like"])
                    user_data[user_id]["validation_hate"].extend(result["hate"])

                # 응답 카운트 증가
                user_data[user_id]["response_count"] += 1

    # Training과 Validation에 모두 존재하는 사용자만 필터링
    common_user_data = {
        user_id: data for user_id, data in user_data.items()
        if (data["training_like"] or data["training_hate"]) and (data["validation_like"] or data["validation_hate"])
    }

    # 응답 수가 많은 상위 100명 선택
    top_users = sorted(common_user_data.items(), key=lambda x: x[1]["response_count"], reverse=True)[:max_users]

    # DataFrame 생성
    df = pd.DataFrame([
        {
            "응답자 ID": user_id,
            "Training 스타일 선호": ", ".join(data["training_like"]),
            "Training 스타일 비선호": ", ".join(data["training_hate"]),
            "Validation 스타일 선호": ", ".join(data["validation_like"]),
            "Validation 스타일 비선호": ", ".join(data["validation_hate"]),
        }
        for user_id, data in top_users
    ])

    return df

# 응답자 선호 데이터 수집 (최대 100명으로 제한)
user_data_df = collect_user_preferences_parallel(training_labels, validation_labels, max_users=100)

# 결과 출력
user_data_df

/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_image에서 유효한 이미지 ID 수: 4066
/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_image에서 유효한 이미지 ID 수: 951
유효한 라벨 파일 수 (/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/training_label): 20304
유효한 라벨 파일 수 (/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/validation_label): 4262


,응답자 ID,Training 스타일 선호,Training 스타일 비선호,Validation 스타일 선호,Validation 스타일 비선호
0,64747,"W_44330_10_sportivecasual_W.jpg, W_39725_19_no...","W_37025_19_lounge_W.jpg, W_38771_10_sportiveca...","W_44330_10_sportivecasual_W.jpg, W_38588_19_ge...","W_11610_90_grunge_W.jpg, W_27828_60_minimal_W...."
1,64561,"W_36601_19_normcore_W.jpg, W_36907_19_genderle...","W_38012_19_normcore_W.jpg, W_42868_10_sportive...","W_38656_10_sportivecasual_W.jpg, W_41448_10_sp...","W_22943_10_athleisure_W.jpg, W_33240_80_bodyco..."
2,64346,"W_28693_10_sportivecasual_M.jpg, W_09289_19_no...","W_25828_10_sportivecasual_M.jpg, W_16822_19_no...","W_09154_50_ivy_M.jpg, W_24103_50_ivy_M.jpg, W_...","W_24931_50_ivy_M.jpg, W_16121_80_bold_M.jpg, W..."
3,63405,"W_17510_19_normcore_M.jpg, W_02890_19_normcore...","W_16919_19_normcore_M.jpg, W_17108_19_normcore...","W_04522_50_ivy_M.jpg, W_15294_50_ivy_M.jpg, W_...","W_12904_50_ivy_M.jpg, W_17443_90_hiphop_M.jpg,..."
4,65139,"W_63644_10_sportivecasual_M.jpg, W_64254_19_no...","W_28911_10_sportivecasual_M.jpg, W_25400_19_no...",W_63644_10_sportivecasual_M.jpg,"W_29942_50_ivy_M.jpg, W_51514_50_ivy_M.jpg, W_..."
...,...,...,...,...,...
95,22324,"W_04553_19_normcore_M.jpg, W_04372_10_sportive...","W_17481_10_sportivecasual_M.jpg, W_16374_10_sp...","W_04636_50_ivy_M.jpg, W_00931_10_sportivecasua...","W_10779_50_ivy_M.jpg, W_04212_50_ivy_M.jpg, W_..."
96,62155,"W_01540_19_normcore_M.jpg, W_31063_19_normcore...","W_33113_19_normcore_M.jpg, W_28203_10_sportive...","W_27854_50_ivy_M.jpg, W_00804_50_ivy_M.jpg, W_...","W_17353_50_ivy_M.jpg, W_00492_50_ivy_M.jpg, W_..."
97,28912,"W_07214_19_normcore_M.jpg, W_01754_10_sportive...","W_16323_10_sportivecasual_M.jpg, W_16846_10_sp...","W_10103_19_normcore_M.jpg, W_16375_80_bold_M.jpg","W_00012_50_ivy_M.jpg, W_04324_90_hiphop_M.jpg,..."
98,35514,"W_05897_10_sportivecasual_M.jpg, W_04539_10_sp...","W_17343_19_normcore_M.jpg, W_06905_19_normcore...","W_06148_50_ivy_M.jpg, W_09154_50_ivy_M.jpg, W_...","W_17796_00_metrosexual_M.jpg, W_16104_60_mods_..."


In [ ]:
# JSON 파일로 저장할 경로 지정
save_path_json = '/content/drive/MyDrive/dataset/2024 데이터 크리에이터 캠프 대학부 데이터셋/2-2 mission/user_data_df_2-2.json'

# JSON 파일로 저장
user_data_df.to_json(save_path_json, orient='records', force_ascii=False)